In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from pathlib import Path
import os
import pandas as pd
dataset_path = Path(r'../input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset')

file_path = list(dataset_path.glob(r'**/*.png'))

# create labels from the folder name
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], file_path))

In [3]:
file_path=pd.Series(file_path).astype(str)
labels=pd.Series(labels)

In [4]:
df = pd.concat([file_path, labels], axis=1)

df.columns = ['image', 'label']

In [5]:
df.label.value_counts()

Black Sea Sprat          1000
Red Sea Bream            1000
Red Sea Bream GT         1000
Trout                    1000
Shrimp GT                1000
Hourse Mackerel GT       1000
Gilt-Head Bream GT       1000
Sea Bass GT              1000
Red Mullet               1000
Red Mullet GT            1000
Trout GT                 1000
Sea Bass                 1000
Gilt-Head Bream          1000
Hourse Mackerel          1000
Striped Red Mullet       1000
Black Sea Sprat GT       1000
Shrimp                   1000
Striped Red Mullet GT    1000
Name: label, dtype: int64

In [6]:
#removing Ground Truth Labels(excess) data
df=df[df['label'].apply(lambda x:x[-2:] !='GT')].reset_index(drop=True)

In [7]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(df,test_size=0.35,random_state=16)
train,val=train_test_split(train,test_size=.15,random_state=16)

In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [9]:
data_generator=ImageDataGenerator(rescale=1/255,height_shift_range=0.2,
                                  width_shift_range=0.2,
                                 rotation_range=40,shear_range=0.2,
                                  zoom_range=0.2,fill_mode='nearest',
                                  horizontal_flip=True)

In [10]:
train=data_generator.flow_from_dataframe(dataframe=train,x_col='image',
                                         y_col='label',color_mode='rgb',
                                         target_size=(200,200),
                                         class_mode='categorical',shuffle=False)
test=data_generator.flow_from_dataframe(dataframe=test,x_col='image',
                                       y_col='label',class_mode='categorical',
                                       target_size=(200,200),color_mode='rgb',
                                       shuffle=False)
val=data_generator.flow_from_dataframe(dataframe=val,x_col='image',
                                       y_col='label',class_mode='categorical',
                                         target_size=(200,200),color_mode='rgb',
                                       shuffle=False)

Found 4972 validated image filenames belonging to 9 classes.
Found 3150 validated image filenames belonging to 9 classes.
Found 878 validated image filenames belonging to 9 classes.


In [11]:
shape=(200,200,3)

In [12]:
model=tf.keras.Sequential([
    tf.keras.layers.Conv2D(128,(3,3),padding='same',input_shape=shape,activation="relu"),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64,(3,3),padding='same',activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(32,(3,3),padding='same',activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(32,(3,3),padding='same',activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dense(16,activation='relu'),
    tf.keras.layers.Dense(9,activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 200, 200, 128)     3584      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 100, 100, 128)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 100, 64)      73792     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 50, 32)        18464     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 25, 25, 32)        9

In [13]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [14]:
cb=tf.keras.callbacks.EarlyStopping(monitor='accuracy',patience=4)
history=model.fit(train,validation_data=val,epochs=20,callbacks=cb)

Epoch 1/20
156/156 [==============================] - 638s 4s/step - loss: 2.1936 - accuracy: 0.1151 - val_loss: 2.0325 - val_accuracy: 0.2005
Epoch 2/20
156/156 [==============================] - 689s 4s/step - loss: 1.9466 - accuracy: 0.2210 - val_loss: 1.6181 - val_accuracy: 0.3542
Epoch 3/20
156/156 [==============================] - 689s 4s/step - loss: 1.5332 - accuracy: 0.3748 - val_loss: 1.2875 - val_accuracy: 0.4601
Epoch 4/20
156/156 [==============================] - 708s 5s/step - loss: 1.3163 - accuracy: 0.4727 - val_loss: 1.1188 - val_accuracy: 0.5638
Epoch 5/20
156/156 [==============================] - 734s 5s/step - loss: 1.1441 - accuracy: 0.5586 - val_loss: 1.1612 - val_accuracy: 0.5843
Epoch 6/20
156/156 [==============================] - 703s 4s/step - loss: 1.0394 - accuracy: 0.5856 - val_loss: 0.9573 - val_accuracy: 0.6458
Epoch 7/20
156/156 [==============================] - 716s 5s/step - loss: 0.9003 - accuracy: 0.6366 - val_loss: 0.7227 - val_accuracy: 0.7312

In [15]:
model.save('fish1.h5')